# Projeto 2 - Classificador Automático de Sentimento

## Ana Beatriz Henriques e Anna Beathriz de Mauro       
<h3>2C - 17/09/2017</h3>

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

In [1]:

import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [2]:
tudo = pd.ExcelFile('bobs1.xlsx')                #Arquivo
dados = pd.read_excel(tudo, 'Treinamento')      #Planilha Treinamento
teste = pd.read_excel(tudo, 'Teste')            #Planilha Teste

<h3> Limpando as mensagens:</h3>

In [3]:
for i in range(len(dados)):
    caract=[',',';','.','!','?',"(",")", ':', '"', "'"]      #caracteres que se desejam retirar
    palavras = dados.Treinamento[i].split()                 # criando lista de listas de palavras em cada mensagem
    letras=[]                                              #lista de lista de letras das palvras
    final=[]                                                #lista de mensagens no final do processo
    for x in palavras:                                      #passando palavra por palavra
        letrasporpalavra=[]                                #lista que irá conter as letras das palavras no for
        for a in range(len(x)):                            #passando letra por letra
            letrasporpalavra.append(x[a])                   #adicionando cada letra na lista
        letras.append(letrasporpalavra)                 #adicionando lista de lista com as letras
    for z in letras:                                    #passando item/lista por lista
        for w in z:                                     #dentro da lista, passando letra por letra
            if w in caract:                            #se estiver entre os caracteres
                d=z.index(w)                           #pega o indice desse caracter
                z[d] = ' '                             #substitui ele na lista por um espaço
    for q in range(len(letras)):                      #q vai passar pelos índices da lista letras (palavra por palavra)
        junto = ''.join(letras[q])                    #junto vai tranformar a lista de letras em palavras           
        if len(junto)!=0: 
            if junto[0] !='@' and junto[0:5]!='https' and junto!= 'rt' and junto[0:3]!='kkk':  #retira alguns outros itens
                final.append(junto)                 #adiciona na lista final
    for a in range(len(final)-1):                    #passando pelo índice de cada palavra
        if final[a] =='de' and final[a+1] == 'bobs':  #juntando de bobs em uma palavra só
            final[a]='debobs'
            final[a+1]=''
    dados.Treinamento[i] = ' '.join(final)         #junta as palavras das mensagens e substitui as mensagens de dados.Treinamento


In [4]:
dados_relevante = dados[dados.relevancia=='r'].Treinamento
dados_irrelevante = dados[dados.relevancia=='i'].Treinamento

In [5]:
rel = []                           #cria lista que ira conter todas as palavras contidas nas mensagens relevantes
for msg in dados_relevante:       #passa mensagem por mensagem
    palavras = msg.split()         #separa as palvras
    for x in palavras:             #adiciona as palavras na lista rel
        rel.append(x)
relevante = pd.DataFrame({"Treinamento":rel}).Treinamento.value_counts()/len(rel)*100   


irrel = []                       #cria lista que ira conter todas as palavras contidas nas mensagens irrelevantes
for msg in dados_irrelevante:    #passa mensagem por mensagem
    palavras = msg.split()       #separa as palvras
    for x in palavras:           #adiciona as palavras na lista rel
        irrel.append(x)
irrelevante = pd.DataFrame({"Treinamento":irrel}).Treinamento.value_counts()/len(irrel)*100


<h3>Calculando porcentagens:</h3>
<ul><li>Porcentagem de mensagens relevantes ou irrelevantes</li></ul>

In [6]:
quantidademsgrelev=0               #cria variável que conterá o número de palavras consideradas relevantes
quantidademsgirrelev=0              #cria variável que conterá o número de palavras consideradas irrelevantes
for i in dados.relevancia:          #passa na coluna relevancia de mwnsagem por mensagem
    if i=='r':                      #se for classificada como relevante adiciona 1 na quantidademsgrelev
        quantidademsgrelev+=1
    if i=='i':                      ##se for classificada como irrelevante adiciona 1 na quantidademsgirrelev
        quantidademsgirrelev+=1
        
quantidademsgrelev=(quantidademsgrelev/len(dados.relevancia)) *100       #tranforma para porcentagem em relação ao total de msg
quantidademsgirrelev=(quantidademsgirrelev/len(dados.relevancia))*100     #tranforma para porcentagem em relação ao total de msg


<ul><li>Número de palavras relevantes ou irrelevantes</li></ul>

In [7]:
rel = []                           #cria lista que ira conter todas as palavras contidas nas mensagens relevantes
for msg in dados_relevante:       #passa mensagem por mensagem
    palavras = msg.split()         #separa as palvras
    for x in palavras:             #adiciona as palavras na lista rel
        rel.append(x)


irrel = []                       #cria lista que ira conter todas as palavras contidas nas mensagens irrelevantes
for msg in dados_irrelevante:    #passa mensagem por mensagem
    palavras = msg.split()       #separa as palvras
    for x in palavras:           #adiciona as palavras na lista rel
        irrel.append(x)


<ul><li>Calculando número de palavras diferentes</li></ul>

In [8]:
palavrasdiferentes=[]                           #cria uma lista que irá conter as palavras diferentes
for palavrarelevante in rel:                    #passa pelas palavras presentes na lista rel(palavras relevantes)
    for palavrairrelevante in irrel:            #passa pelas palavras presentes na lista irrel(palavras irrelevantes)
        if palavrarelevante!= palavrairrelevante:       # se as palavras forem diferentes
            if palavrarelevante not in palavrasdiferentes:           # se a palavra não estiver em palavrasdiferentes, adiciona
                palavrasdiferentes.append(palavrarelevante)
            if palavrairrelevante not in palavrasdiferentes:
                palavrasdiferentes.append(palavrairrelevante)            

<ul><li>Calculando probabilidade das palavras dado relevante ou irrelevante</li></ul>

In [9]:
#cria dicionário com a porcentagem das palavras
relevante = pd.DataFrame({"Treinamento":rel}).Treinamento.value_counts()/(len(rel)+len(palavrasdiferentes)) + (1/(len(rel)+len(palavrasdiferentes)))
relevante=relevante*100
irrelevante = pd.DataFrame({"Treinamento":irrel}).Treinamento.value_counts()/(len(irrel)+len(palavrasdiferentes)) + (1/(len(irrel)+len(palavrasdiferentes)))
irrelevante=irrelevante*100


<h2>Planilha Teste</h2>

<h3> Limpando as mensagens:</h3>

In [10]:
for i in range(len(teste)):
    caract=[',',';','.','!','?',"(",")", ':', '"', "'"]      #caracteres que se desejam retirar
    palavras = teste.Teste[i].split()                 # criando lista de listas de palavras em cada mensagem
    letras=[]                                              #lista de lista de letras das palvras
    final=[]                                                #lista de mensagens no final do processo
    for x in palavras:                                      #passando palavra por palavra
        letrasporpalavra=[]                                #lista que irá conter as letras das palavras no for
        for a in range(len(x)):                            #passando letra por letra
            letrasporpalavra.append(x[a])                   #adicionando cada letra na lista
        letras.append(letrasporpalavra)                 #adicionando lista de lista com as letras
    for z in letras:                                    #passando item/lista por lista
        for w in z:                                     #dentro da lista, passando letra por letra
            if w in caract:                            #se estiver entre os caracteres
                d=z.index(w)                           #pega o indice desse caracter
                z[d] = ' '                             #substitui ele na lista por um espaço
    for q in range(len(letras)):                      #q vai passar pelos índices da lista letras (palavra por palavra)
        junto = ''.join(letras[q])                    #junto vai tranformar a lista de letras em palavras           
        if len(junto)!=0: 
            if junto[0] !='@' and junto[0:5]!='https' and junto!= 'rt' and junto[0:3]!='kkk':  #retira alguns outros itens
                final.append(junto)                 #adiciona na lista final
    for a in range(len(final)-1):                    #passando pelo índice de cada palavra
        if final[a] =='de' and final[a+1] == 'bobs':  #juntando de bobs em uma palavra só
            final[a]='debobs'
            final[a+1]=''
    teste.Teste[i] = ' '.join(final)         #junta as palavras das mensagens e substitui as mensagens de teste.Teste

<h3>Classificando:</h3>

In [11]:
teste.loc[: , 'Diferença de probabilidades']='x'           #cria coluna Diferença de probabilidades
teste.loc[: , 'Classificação Digital']='z'                 # cria coluna Classificação Digital
for i in range(len(teste)):                                   #passa por msg de teste
    palavras = teste.Teste[i].split()                         #separa por palavra
    probabilidaderelev =quantidademsgrelev           #probabilidade da frase ser relevante/irrelevante que tem valor inicial da
    probabilidadeirrelev=quantidademsgirrelev        #probabilidade de uma palavra ser relevante/irrelevante no geral
    for q in palavras:
        if q in relevante:                             #se a palavra já existir, pega mutiplica a probabilidaderelev pela
            probabilidaderelev*=relevante[q]           #probabilidade de aparecer essa palavra dado relevante
        else:
            probabilidaderelev*=1/(len(rel)+len(palavrasdiferentes))   #se a palavra não existir a probabilidade atribuída a ela é essa
        if q in irrelevante:                               #Repete-se o processo
            probabilidadeirrelev*= irrelevante[q]
        else:
            probabilidadeirrelev*= 1/(len(irrel)+len(palavrasdiferentes))
    if probabilidaderelev> probabilidadeirrelev:   #verifica qual probabilidade é maior
        teste['Classificação Digital'][i]='r'      # se relevante, classicfica como r
    else:
        teste['Classificação Digital'][i]='i'      #se irrelevante, classifica como i
    probdifR=probabilidaderelev-probabilidadeirrelev   #calcula a diferença entre as probabilidades
    
    teste['Diferença de probabilidades'][i]=probdifR   #aribui o valor dessa diferença na coluna Diferença de probabilidades
#Classifica de modo mais específico em função dessa diferença:
teste.loc[teste['Diferença de probabilidades'] < - (10**(-10)), 'Especifico']='mi'    #muito irrelevante
teste.loc[teste['Diferença de probabilidades'] > - (10**(-10)), 'Especifico']='i'     #irrelevante
teste.loc[teste['Diferença de probabilidades'] > - (10**(-20)), 'Especifico']='n'     #neutro
teste.loc[teste['Diferença de probabilidades'] >  (10**(-20)), 'Especifico']='r'      #relevante
teste.loc[teste['Diferença de probabilidades'] >  (10**(-10)), 'Especifico']='mr'     #muito relevante



___
## Verificando a performance

Considerando:
* Positivos verdadeiros como marcado pelo Classificador como relevante e são relevantes
* Positivos falsos como marcado pelo Classificador como relevante e não são relevantes
* Negativos verdadeiros como marcado pelo Classificador como não relevante e não são relevantes
* Negativos falsos como marcado pelo Classificador como não relevante e são relevantes

In [12]:
pv=0     #positivo verdadeiro
pf=0     #positivo falso
nv=0     #negativo verdadeiro
nf=0     #negativo falso
for i in range(len(teste['Classificação Digital'])):      #passa por cada linha da coluna Classificação Digital
    if teste['Classificação Digital'][i] == 'i':          #se a classificação difital for irrelevante:
        
        if teste.Manual[i] == 'i':                               #e a manual também, portanto é negativo verdadeiro
            nv+=1                                                #então se adiciona 1 ao número de negativos verdadeiros
            
        else:                                                    #e a manual não, é negativo falso
            nf+=1                                                #então se adiciona 1 ao número de negativos falsos
            
    if teste['Classificação Digital'][i] == 'r':          #se a classificação difital for relevante:
        
        if teste.Manual[i] == 'r':                               #e a manual também, portanto é positivo verdadeiro
            pv+=1                                                #então se adiciona 1 ao número de positivos verdadeiros
             
        else:                                                    #e a manual não, é positivo falso
            pf+=1                                                #então se adiciona 1 ao número de positios falsos
            
#Transformando em porcentagens:
pf=(pf/len(teste['Classificação Digital']))*100
pv=(pv/len(teste['Classificação Digital']))*100
nv=(nv/len(teste['Classificação Digital']))*100
nf=(nf/len(teste['Classificação Digital']))*100

print('A porcentagem de positivos verdadeiros é de:           {}%'.format(pv))
print('A porcentagem de positivos falsos é de:                {}%'.format(pf))
print('A porcentagem de negativos verdadeiros é de:           {}%'.format(nv))
print('A porcentagem de negativos falsos é de:                {}%'.format(nf))


A porcentagem de positivos verdadeiros é de:           62.0%
A porcentagem de positivos falsos é de:                20.0%
A porcentagem de negativos verdadeiros é de:           14.499999999999998%
A porcentagem de negativos falsos é de:                3.5000000000000004%


<h3>Classificação mais específica</h3>

Considerando:
* Muito relevantes como mr
* Relevantes como r
* Neutros como n
* Irrelevantes como i
* Muito irrelevantes como mi

Além da utilização do classificador, classificou-se as mensagens manualmente utilizando os seguintes critérios:
* Muito relevantes como aquelas em que se há uma crítica negativa
* Relevantes como aquelas em que há críticas positivas
* Neutros como como aquelas que continham informações como a confirmação da presença no fast-food
* Irrelevantes como aquelas que se referiam ao 'Bobs' de maneira geral, junto com vários fast-foods, porém sem fazer uma crítica
* Muito irrelevantes como aquelas que não se referiam realmente ao 'Bobs'


In [13]:
manxdig=pd.crosstab(teste.Especifico,teste.EspManual,margins=True,rownames=['Digital'] , colnames=['Manual'], normalize=True)*100
print('\033[1m'+'  Classificação Manual X Digital')
manxdig

  Classificação Manual X Digital


Manual,i,mi,mr,n,r,All
Digital,,,,,,
i,0.5,6.0,0.5,0.5,1.0,8.5
mi,0.5,4.0,0.5,0.0,0.5,5.5
mr,1.0,6.0,3.0,17.0,19.0,46.0
n,1.0,9.5,0.0,5.0,2.5,18.0
r,2.5,4.5,2.0,6.5,6.5,22.0
All,5.5,30.0,6.0,29.0,29.5,100.0


<em><b>Tabela 1:</b> Tabela comparando a classificção específica manual e a feita pelo Classificador</em>

In [14]:
porcacerto = manxdig.i.i+manxdig.mi.mi+manxdig.r.r+manxdig.mr.mr +manxdig.n.n
print("A probabilidade de acerto do classificador, nesse caso é de {}%, sendo considerada muito baixa.".format(porcacerto))


A probabilidade de acerto do classificador, nesse caso é de 19.0%, sendo considerada muito baixa.


<p style = "text-indent:4em"> A partir da análise da <em><b>Tabela 1</b></em>, percebe-se que a probabilidade de erro do Classificador torna-se muito grande. Isso ocorre, pois, no nosso contexto atual, a separação entre muito relevante, relevante, neutro, irrelevante e muito irrelevante não fica muito consistente, pois torna-se muito subjetiva para que, apenas com a frequência das palavras se possa tirar conclusões.</p>
<p style = "text-indent:4em">Outro fator importante que causa a inconsistência é que o padrão de classificação precisaria ser mudado. Isso ocorre devido ao fato de que, em um primeiro momento, considerou-se na classificação apenas o relacionado a marca ou não e, para uma classificação mais específica, seria necessário considerar o nível de relevância, o qual poderia ser entendido como a classificação do que se fala da marca, o que somente seria lógico se fosse feito apenas com as consideradas anteriormente como relevantes.</p>

___
## Concluindo



<style = 'text-ident:4em'> No geral o classificador possui pontos positivos e negativos. Vale notar que no trabalho apenas classificamos como relevante twwets intimamente relacionados ao produto escolhido frente a quantidade significativa de tweets que utilizavam o nome procurado, mas que discorriam sobre outros assuntos irrelevantes para a pesquisa.
 Frente ao exposto, dentre os aspectos positivos, tem-se que o classificador esta treinado para entender a frase "de bobs", considerada uma gíria, como uma única palavra. Neste contexto, diminui-se a possibilidade de equívocos na classificação, posto que, é retirada a interferência da gíria na frequência da palavra do produto procurado ("bobs"). Outro aspecto positivo está na capacidade de considerar e classificar atributos ausentes na base de treinamento.
    Entretanto, dentre os aspectos negativos, deve-se notar que por levar apenas a frequência das palavras de maneira independente em consideração, ou seja ignorando o significado da relação entre elas em uma frase, o classificador pode vir a cometer equívocos. Neste ínterim, sobretudo no caso de no futuro classificarmos os dados relevantes em comentários que elogiam ou denigrem o produto, levar em consideração a correlação entre as palavras torna-se amplamente necessário. Outro ponto importante é que o classificador necessita de supervisão, ou seja, de uma base de treinamento para servir de comparação na análise de amostragens desconhecidas e assim fazer previsões. Desse modo, é mister notar que não se pode usar automaticamente o próprio classificador na base de treinamento aplicado a novos tweets. Além disso, uma vez que o classificador depende de uma base já classificada, a quantidade reduzida de tweet manualmente rotulados também interfere no desempenho. 


<p style = "text-indent:4em">Dentre outras utilidades do Classificador Naive-Bayes, tem-se a detecção de fraudes, filtragem de spam em e-mails, análise de risco para créditos bancários e diagnósticos médicos. No que diz respeito às fraudes e aos spams, em ambos o classificador naive-bayers se torna pertinente pois possui a capacidade de aprender frente as atualizações constantes desse contexto. Na questão do crédito bancário, é possível classificar o cliente, por exemplo, como inadimplente mediante seus dados cadastrais. Por fim, para os diagnósticos médicos, analisa-se a probabilidade do paciente possuir uma doença específica dado um ou mais sintomas determinados.</p>
<p style = "text-indent:4em">No geral, o classificador realizado neste projeto possui pontos positivos e negativos. Vale notar que no trabalho apenas classificou-se como relevante tweets intimamente relacionados ao produto escolhido, a marca <em>"Bobs"</em>, frente a quantidade significativa de tweets que utilizavam o nome procurado, mas que discorriam sobre outros assuntos irrelevantes para a pesquisa.</p>
<p style = "text-indent:4em">Diante do exposto, dentre os aspectos positivos, tem-se que o classificador está treinado para entender a expressão "de bobs", considerada uma gíria, como uma única palavra. Neste contexto, diminui-se a possibilidade de equívocos na classificação, posto que é retirada a interferência da gíria na frequência da palavra do produto procurado ("bobs"). Outro aspecto positivo está na capacidade de considerar e classificar atributos ausentes na base de treinamento.</p>
<p style = "text-indent:4em">Além disso, o resultado do classificador, por mais que houve um erro de aproximadamente 24%, pode ser considerado como algo positivo, já que a porcentagem de negativos falsos(3.5%) é bem menor do que a de positivos falsos(20%). Ou seja, no contexto de uma empresa, seria melhor para ela receber positivos falsos do que negativos falsos, pois, por mais que será necessário mais mão-de-obra pois haverão mais tweets considerados relevantes e não o são, a probabilidade de que não se note um tweet importante é menor.</p>
<p style = "text-indent:4em">Entretanto, dentre os aspectos negativos, deve-se notar que, por levar em consideração apenas a frequência das palavras de maneira independente, ou seja, ignorando o significado da relação entre elas em uma frase, o classificador pode vir a cometer equívocos. Entretanto, para a classificação feita, o sentido real das mensagens não foi tão crucial não sendo necessário identificar dupla negação ou sarcasmo quanto caso se quisesse classificar os dados relevantes em comentários que elogiam ou denigrem o produto,o que, para essa expansão, levar em consideração a correlação entre as palavras torna-se amplamente necessário, já que o que será analisado será o sentido.</p>
<p style = "text-indent:4em">Entretanto, o computador não consegue interpretar subjetivamente as mensagens mesma maneira que pessoas, o que dificulta o processo de análise das ocorrências de sarcasmo e dupla negação. Mas, a porcentagem de tweets com esses fatores provavelmente será pequena, não alterando muito os resultados do Classificador. Porém, em épocas como quando o "#sóquenão" e suas variações estavam na moda, em que, normalmente se escrevia uma coisa contrária ao que realmente se queria, o processo de análise torna-se muito mais complexo.  </p>
<p style = "text-indent:4em">Outro ponto importante é que o classificador necessita de supervisão, ou seja, de uma base de treinamento para servir de comparação na análise de amostragens desconhecidas e assim fazer previsões. Desse modo, é mister notar que não se pode alimentar automaticamente a base de Treinamento com a classificação realizada pelo Classificador em novos tweets, principalmente pois não será possível saber se a classificação feita nessas novas mensagens estarão corretas, o que poderia acarretar em uma alimentação equívoca do Classificador, o que afetaria o seu desempenho. Por isso, uma vez que o classificador depende de uma base já classificada, a quantidade reduzida de tweets manualmente rotulados também interfere no desempenho.</p>

<p style = "text-indent:4em"> Em busca de possíveis melhorias para o Classificador, encontrou-se quatro maneiras:</p>
<ol><em>
    <li>N-grams</li>
    <li>Tf-Idf</li>
    <li>Stem Extraction</li>
    <li>Levenshtein Distance</li>
</em></ol>

<h4>I) N-grams:</h4>
<p style = "text-indent:4em">Levar em consideração conjuntos de palavras e suas correlações na frase. Dessa maneira, caso se encaminhe a proposta de classificar em elogios ou desmerecimentos, estabelecer diferença de significado entre certas frases tais como “muito bom” e “muito ruim” torna-se substancial. Objetivando aplicar este método deve -se aplicar condições no código nos quais a palavra dependerá das posições das “n” palavras anteriores.</p>

<h4>II) Tf-Idf (Frequência do termo inverso da frequência nos documentos):</h4>
<p style = "text-indent:4em">A importância de uma palavra aumenta de acordo com o número de ocorrências durante a classificação, porém esse número é equilibrado considerando a frequência desta mesma palavra nos dois grupos de relevância. Caso sua frequência seja alta em ambos significa que sua importância na classificação deveria ser reduzida, esse é o caso de palavras costumeiramente presentes no decorrer de uma frase tais como artigos. No código deve-se calcular a frequência em cada classificação (Tf) e calcular a presença em ambos (Idf) multiplicando os dois resultados obtendo-se a frequência equilibrada da palavra(Tf-Idf).</p>

<h4>III) Stem Extraction:</h4>
<p style = "text-indent:4em">Extrair o radical das palavras para analisar o seu sentido principal. Reunindo as diferentes maneiras de se escrever as palavras como com diminutivo e plural, classificando "balas" e "bala" da mesma maneira, já que, essencialmente, são iguais. Na <em><b>Figura 1</b></em> está presente esse processo de extração do radical de uma palavra.</p>
<img src = 'StemExtraction.png' width=800px/>
<em><b>Figura 1:</b> Extração do radical de uma palavra. Fonte:http://wsl.softwarelivre.org/2005/0020/20.pdf</em>

<h4>IV) Levenshtein Distance</h4>
<p style = "text-indent:4em">Se refere ao número mínimo de operações necessárias para transformar um string em outro. Sendo consideradas como operações a substituição, adição ou subtração de letras. Sua utilização é muito importante em corretores ortográficos,reconhecimento de fala,análise de DNA e detecção de plágio, por exemplo. Ainda assim, há casos em que pode falhar, como analisando "mata" e "mala", que tem uma distância 1, porém não tem relamente semelhança semântica.</p>

<h4>Matriais de Pesquisa e fontes:</h4>
<ol>
    <li> N-grams
        <ul>
            <li>https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf  </li>
           
        </ul>
    </li>
    
    <li>Tf-Idf
        <ul>
            <li>http://www.tfidf.com/ </li>
        </ul>
    </li>
    <li> Stem Extraction
        <ul>
            <li>http://wsl.softwarelivre.org/2005/0020/20.pdf</li>
        </ul>
    </li>
    
    <li> Levenshtein Distance
    <ul>
        <li>https://people.cs.pitt.edu/~kirk/cs1501/Pruhs/Spring2006/assignments/editdistance/Levenshtein%20Distance.htm</li>
        <li>https://pt.wikipedia.org/wiki/Dist%C3%A2ncia_Levenshtein</li>
    </ul>
    </li>
    <li> Outros
        <ul>
            <li>https://link.springer.com/article/10.1023%2FA%3A1009976227802?LI=true</li>
            <li>http://www.facom.ufu.br/~backes/pgc204/Aula05-ClassificadoresBaeysianos.pdf</li>
            <li>http://www.teses.usp.br/teses/disponiveis/3/3142/tde-25052009-162507/publico/Dissertacao_Cristiane_Karcher_revisada.pdf</li>
            <li>http://www.lbd.dcc.ufmg.br/colecoes/erbd/2013/0019.pdf</li>
            <li>http://www.lbd.dcc.ufmg.br/colecoes/eniac/2016/059.pdf</li>
        </ul>
    </li>
</ol>    